In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

plt.rcParams['text.usetex'] = True

# Functions

In [2]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]


# Groups, Routes to Edges:

In [3]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')

# df_od_flow_data
# df_data

In [4]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])


In [5]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 612.539616936298,
  'Demand (eligible group, 1)': 47.77809012103125,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 27.564282762133406,
  'VoT (eligible group, 2)': 0.10238603988603988,
  'Demand (ineligible group, 1)': 115.15744798402402,
  'VoT (ineligible group, 1)': 0.2754407051282051,
  'Demand (ineligible group, 2)': 134.75871572598555,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 287.28108034312373,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 98.03491986336807,
  'Demand (eligible group, 1)': 7.64672374934271,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 4.411571393851563,
  'VoT (eligible g

In [6]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [7]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

# od_to_edges_array

In [8]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array[od_index, group_index] = od_value[VoT_name]

# print(demand_array)
# VoT_array

# Download Latency Parameters Data

In [9]:
directory_path_latency = '../data/pems_latency_inference___101_N_Sep_to_Nov_2024/'
df_latency_params = pd.read_csv(directory_path_latency + 'latency_params.csv')

# list(df_latency_params.loc[:, "Palo Alto"])

In [10]:
dict_latency_params = {}

city_list = list(df_latency_params.columns)[1:]

for city in city_list:
#     if city != "Belmont":
    if 1 == 1:
        dict_latency_params[city] = {}
        dict_latency_params[city]["Flow (at bend)"] = df_latency_params.loc[:, city][0]
        dict_latency_params[city]["Time (at bend)"] = df_latency_params.loc[:, city][1]
        dict_latency_params[city]["Slope (after bend)"] = df_latency_params.loc[:, city][2]

dict_latency_params

{'Palo Alto': {'Flow (at bend)': 861.9884999999999,
  'Time (at bend)': 1.326448252485291,
  'Slope (after bend)': 0.0007826659300288},
 'East Palo Alto': {'Flow (at bend)': 1001.5178571428572,
  'Time (at bend)': 2.21312655334384,
  'Slope (after bend)': 0.0005844841481716},
 'Redwood City': {'Flow (at bend)': 881.1846666666667,
  'Time (at bend)': 4.892192374894639,
  'Slope (after bend)': 0.0015637240321908},
 'Belmont': {'Flow (at bend)': 1278.948125,
  'Time (at bend)': 1.1999111785887613,
  'Slope (after bend)': 0.0019941383648286},
 'San Mateo': {'Flow (at bend)': 1034.0928260869566,
  'Time (at bend)': 5.541006284392742,
  'Slope (after bend)': 0.0021472617019578},
 'Burlingame': {'Flow (at bend)': 845.15,
  'Time (at bend)': 1.5031113452842335,
  'Slope (after bend)': 0.0003066013881685},
 'Millbrae': {'Flow (at bend)': 853.1818181818181,
  'Time (at bend)': 2.384328451750197,
  'Slope (after bend)': 0.0003218564565735}}

# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [11]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [12]:
# np.array([1, 2, 3]) * np.array([6, 5, 3])

In [17]:
def coeff_from_input(coeff_input, num_edges, num_gp_lanes):
    # Goal: Differentiate between express and general purpose lanes.
    
    assert np.all(coeff_input >= 0.0), "All entries of coeff_input must be non-negative."
    assert len(coeff_input.shape) == 2, "coeff_input should be a matrix, i.e., 2-D array"
    assert coeff_input.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    assert coeff_input.shape[1] == num_edges, "Latency functions should be defined across all edges."
    
    coeff_length = 3
    
    ex_to_gp_multiplier = np.array([1, 1/num_gp_lanes, num_gp_lanes]).reshape((coeff_length, 1)) \
                            @ np.ones((1, num_edges))
    
    coeff_full = np.zeros((coeff_length, num_edges, 2))
    coeff_full[:, :, 0] = coeff_input
    coeff_full[:, :, 1] = coeff_input * ex_to_gp_multiplier
    
    return coeff_full

def latency_max(flow_max, coeff):
    
    assert np.all(coeff >= 0.0), "coeff should be non-negative"
    assert len(coeff.shape) == 1, "coeff should be a 1-D array."
    assert coeff.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    
    return coeff[0] + max(coeff[1] * (flow_max - coeff[2]), 0)

def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
                demand_array, VoT_array, num_el, od_to_edges_array, y, \
                coeff_input):

    coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for e in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
            
    print()
    print("in_indices:", in_indices)
    print("el_indices:", el_indices)
    print()
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            for k in range(2):
                ell[e, k, t] = coeff[0, e, k] + coeff[1, e, k] * max(x[e, k, t] - coeff[2, e, k], 0)

    obj_E = sum( y[(od, g, e, 0, t)] * VoT_array[od, g] * ell[e, 0, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * (VoT_array[od, g] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 2, t)] * VoT_array[od, g] * ell[e, 1, t] \
                  for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in el_indices for t in range(T) )
    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g] * ell[e, 1, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in in_indices for t in range(T) )
    obj_R = sum( y[(od, g, e, 0, t)] * tau[e, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) )

    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare, obj_E, obj_R, obj_I


# Latency, total, throughout the entire time horizon. 

def latency_total(T, num_edges, lambda_E, lambda_R, lambda_I, tau, \
                  demand_array, VoT_array, num_el, y, od_to_edges_array, \
                  coeff_input):
    
    coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges' second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for edge in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]
    
    # In full:
    # y indices: (od, group, edge, "lane", time)

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            for k in range(2):
                ell[e, k, t] = coeff[0, e, k] + coeff[1, e, k] * max(x[e, k, t] - coeff[2, e, k], 0)

    latency = sum( x[e, k, t] * ell[e, k, t] \
                for e in range(num_edges) for k in range(2) for t in range(T) )

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return latency

In [18]:
# arr = np.arange(5)
# arr.reshape((5, 1))

In [19]:
def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max, B_max = 1.0, B_min = 0.1):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
    assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
    tau_feas = cp.Variable((num_edges, T))
#     B_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau)

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max.reshape((num_edges, 1)) * np.ones((1, T))]
#     constraints += [B_feas <= B_max]

#     constraints += [B_feas <= sum(tau_feas[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
#                     for od in range(len(od_to_edges_list_full))]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

#     print()
#     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
#     print()

    B_max_list = [sum(tau_feas.value[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
                            for od in range(len(od_to_edges_list_full))]
#     B_max_wrt_tau_feas = min([B_max for B_max in B_max_list if B_max >= B_min])
    B_max_wrt_tau_feas = max(B_max_list)
    B_feas = min(B, B_max_wrt_tau_feas)
    
    print()
    print("B:", B)
    print("B_max_wrt_tau_feas:", B_max_wrt_tau_feas)
    print("B_feas:", B_feas)
    print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
            np.round(np.maximum(B_feas, 0.0), decimals=4)
    
#     return tau_feas.value, B_feas



In [20]:
# # OLD def proj_tau_B:

# def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max, B_max = 1.0):

# #     print()
# #     print("tau.shape[0]:", tau.shape[0])
# #     print("num_edges:", num_edges)
# #     print()

#     assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
#     assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
#     tau_feas = cp.Variable((num_edges, T))
#     B_feas = cp.Variable(1)
    
#     func = cp.sum_squares(tau_feas - tau) + (B_feas - B)**2

#     objective = cp.Minimize(func)

#     constraints = []
#     constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
#     constraints += [tau_feas <= tau_max.reshape((num_edges, 1)) * np.ones((1, T))]
#     constraints += [B_feas <= B_max]

#     constraints += [B_feas <= sum(tau_feas[e, t] for e in od_to_edges_list_full[od]) \
#                     for od in range(len(od_to_edges_list_full)) for t in range(T)]
    
#     prob = cp.Problem(objective, constraints)
#     result = prob.solve()

# #     print()
# #     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
# #     print()

#     return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
#             np.round(np.maximum(B_feas.value, 0.0), decimals=4)

# Chinmay's Algorithm:

## Convex Program for CBCP and DBCP Equilibria:

In [21]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [22]:
# num_edges = 8
# od_to_edges_array = np.array([[0, 0], [0, 3], [0, 7], [1, 5], [1, 7], [3, 3], [3, 6]])

# od_to_edges_list_full = [list(range(od_to_edges_array[od, 0], od_to_edges_array[od, 1] + 1)) \
#                       for od in range(od_to_edges_array.shape[0])]

# edge_to_od = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
#             for e in range(num_edges)]

# print(od_to_edges_list_full)
# print()
# print(edge_to_od)

In [23]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0

def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
                      demand_array, VoT_array, num_el, coeff_input):
    
#     print("tau (in solve_CBCP_direct):", tau)
    
    coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)
    
    assert demand_array.shape == VoT_array.shape, "demand_array and VoT_array should have identical shape."
    assert np.all(demand_array > 0.0), "Each entry of demand_array must be strictly positive."
    assert np.all(tau >= 0.0), "Each entry of tau must be non-negative."
    assert num_el <= demand_array.shape[1], "num_el, the number of eligible income groups, should not exceed \
                                            demand_array.shape[1], which should equal the number of income groups."
    
    num_groups = demand_array.shape[1]
    
    ## Variable indices:
    # y indices: (od, income group, edge, "lane", time)
    # y indices: (od, income group, edge, "lane", time)

    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    group_indices = list(range(num_groups))
    
    od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                          for od in range(od_to_edges_array.shape[0])]
    
    edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
                   for e in range(num_edges)]
    
    num_od = len(od_to_edges_list_full)
    
    # Variables:
    y = {}
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
                for g in in_indices:
                    for k in [0, 1]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
    
    x = {}
    for e in range(num_edges):
        for k in [0, 1]:
            for t in range(T):
                x[(e, k, t)] = cp.Variable(1)

    # Objective:
    func = 0.0
    for e in range(num_edges):
        for k in range(2):
            for t in range(T):
                func += coeff[0, e, k] * x[e, k, t] \
                        + 0.5 * coeff[1, e, k] * cp.square(cp.maximum(x[e, k, t] - coeff[2, e, k], 0))
    
#     print("VoT_array.shape:", VoT_array.shape)
#     print("el_indices:", el_indices)
#     print("in_indices:", in_indices)
    
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g]
                for g in in_indices:
                    func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g]

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in el_indices  \
                    for k in [0, 1, 2] for t in range(T)]
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in in_indices  \
                    for k in [0, 1] for t in range(T)]
    constraints += [x[(e, k, t)] >= 0.0 for e in range(num_edges) \
                    for k in [0, 1] for t in range(T)]
    
#     print()
#     print("demand_array:", demand_array)
#     print()
#     print("B:", B)
#     print()
#     print("el_indices:", el_indices)
#     print()
#     print("in_indices:", in_indices)
#     print()
#     print("edge_to_ods:", edge_to_ods)
#     print()
#     print("od_to_edges_list_full:", od_to_edges_list_full)
#     print()
    
    od_g_e_t_list = []

    for e in range(num_edges):
        for t in range(T):
            
            ## Edge contraints:
            constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 0, t)] ]
            constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 1, t)] ]

            ## Group flow constraints:
            for od in edge_to_ods[e]:
                for g in el_indices:
#                     print("(od, g, e, t):", od, g, e, t)
                    assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
                    od_g_e_t_list += [(od, g, e, t)]
        
#                     if (od, g, e, t) == (0, 0, 0, 0):
#                     if (e, t) == (2, 0) or (2, 1) or (2, 3):
#                         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g]]
                    
#                 for g in in_indices:
# #                     print("(od, g, e, t):", od, g, e, t)
#                     assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
#                     od_g_e_t_list += [(od, g, e, t)]
                    
#                     constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g]]

            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in el_indices]
            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in in_indices]
    
    constraints += [sum(y[(od, g, e, 0, t)] * tau[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
                    <= B * demand_array[od, g] for od in range(num_od) for g in el_indices]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
#     result = prob.solve(solver = cp.MOSEK, verbose = True)
    result = prob.solve(solver = cp.MOSEK)
    
#     for variable in prob.variables():
#         print("Variable %s" % variable.name())
    
    assert prob.status != "infeasible", "problem.status should not be infeasible."
    assert prob.status != "unbounded", "problem.status should not be unbounded."
    print()
    print("prob.status:", prob.status)

    # Extract Values:
    y_values = {}
    for e in range(num_edges):
        for od in edge_to_ods[e]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
                for g in in_indices:
                    for k in [0, 1]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)

    return y_values


## <font color='blue'>Implement Zeroth-order Gradient Descent:</font> 

In [24]:
## Functions defined above:

# def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max = 1.0, B_max = 1.0):

# def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):

#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):

In [61]:
# coeff_input

array([[8.61988500e+02, 1.00151786e+03, 8.81184667e+02, 1.27894812e+03,
        1.03409283e+03, 8.45150000e+02, 8.53181818e+02],
       [7.82665930e-04, 5.84484148e-04, 1.56372403e-03, 1.99413836e-03,
        2.14726170e-03, 3.06601388e-04, 3.21856457e-04],
       [1.32644825e+00, 2.21312655e+00, 4.89219237e+00, 1.19991118e+00,
        5.54100628e+00, 1.50311135e+00, 2.38432845e+00]])

In [26]:
time_1 = time.time()

T = 5

num_el = 2

assert demand_array.shape == VoT_array.shape, "demand_array and VoT_array should have the same shape."

num_ods = demand_array.shape[0]
group_indices = list(range(demand_array.shape[1]))
# num_edges = 6
num_edges = 7
num_gp_lanes = 3

# tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4, 0.5])
# assert tau.shape[0] == num_edges, "tau must have a num_edges number of entries."

# B = 0.8
a = np.array([1E-3, 1E-3, 0.0, 0.0, 0.0])

error_bound = 1E-2
diffs_num_cols = 5

# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_E, lambda_R, lambda_I = 1.0, 2.0, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

demand_edges_array = np.zeros(num_edges)

# demand_array_temp = np.ones(demand_array.shape)
# demand_array_temp = np.random.uniform(low=0.05, high=0.5, size=demand_array.shape)
demand_array_temp = demand_array


# TODO: Modify coeff_input:

## coeff_input: const, slope, x-coordinate of transition point

# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))
# coeff_input = np.zeros((3, num_edges))

counter = 0
for city in dict_latency_params.keys():
    coeff_input[0, counter] = dict_latency_params[city]["Flow (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Time (at bend)"]
    
    counter += 1


# def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):
# Already defined: T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array

    
# print("od_to_edges_array[1, 0]:", int(od_to_edges_array[1, 0]))
# print("range(od_to_edges_array[1, 0], od_to_edges_array[1, 1] + 1):",\
#      range(int(od_to_edges_array[1, 0]), int(od_to_edges_array[1, 1]) + 1))

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                         for od in range(num_ods)]
edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
               for e in range(num_edges)]
for e in range(num_edges):
    demand_edges_array[e] = sum([np.sum(demand_array_temp[od, :]) for od in range(num_ods) \
                                 if od in edge_to_ods[e]])

tau_max_from_latency = np.zeros(num_edges)
for e in range(num_edges):
    VoT_max_e = max([VoT_array[od_index, group_index] for od_index in edge_to_ods[e] \
                     for group_index in range(num_groups_per_od) ])
    tau_max_from_latency[e] = VoT_max_e * (latency_max(demand_edges_array[e], coeff_input[:, e]) - coeff_input[0, e]) \
                                / num_gp_lanes

tau_max_monetary_value = 15.0
# fraction_tau_max = tau_max_monetary_value / np.max(tau_max_from_latency)

tau_upper_limit = np.minimum(tau_max_from_latency, tau_max_monetary_value)
B_upper_limit = max([sum(tau_upper_limit[e] for e in od_to_edges_list_full[od]) \
                     for od in range(len(od_to_edges_list_full))]) * T

d = num_edges * T + 1
# num_iters = 1000
# num_iters = 100
num_iters = 140
# num_iters = 3
tau = np.zeros((num_edges, T, num_iters))
tau_perturbed = np.zeros((num_edges, T, num_iters))

B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)
delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 1E-5
delta_bar = 0.02

welfare_list = []

fraction_tau_max = 0.5
fraction_B_max = 0.5

# tau[:, :, 0] = fraction_tau_max * tau_upper_limit.reshape((num_edges, 1)) @ np.ones((1, T))
tau_upper_limit_array = tau_upper_limit.reshape((num_edges, 1)) @ np.ones((1, T))
tau[:, :, 0] = np.random.triangular(np.zeros((num_edges, T)), tau_upper_limit_array * fraction_tau_max, tau_upper_limit_array)
# B[0] = fraction_B_max * B_upper_limit
B[0] = np.random.triangular(0.0, fraction_B_max * B_upper_limit, B_upper_limit)

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1 )) \
                         for od in range(od_to_edges_array.shape[0])]

# print("od_to_edges_list_full:", od_to_edges_list_full)
# print("tau_max_from_latency:", tau_max_from_latency)
# print("B_max", B_max)


## To disable when restarting from scratch
# tau[:, :, 0] = tau_next_init
# B[0] = B_next_init
# eta[0] = eta_bar * (index_next_init+1)**(-1/2) * d**(-1)
# delta[0] = delta_bar * (index_next_init+1)**(-1/4) * d**(-1/2)

for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(d)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
#     print("w_i:", w_i)
    tau_perturbed[:, :, i] = tau[:, :, i] + delta[i] * w_i[:-1].reshape((num_edges, T))
    B_perturbed[i] = B[i] + delta[i] * w_i[-1]
    
#     if tau_perturbed[:, :, i] < B_perturbed[i] or tau_perturbed[:, :, i] < 0 or B_perturbed[i] < 0:

    print("i:\n", i)
    print("tau[:, :, i], before projection:\n", tau[:, :, i])
    print("B[i], before projection:\n", B[i])
    print("tau_perturbed[:, :, i], before projection:\n", tau_perturbed[:, :, i])
    print("B_perturbed[i], before projection:\n", B_perturbed[i])

    tau_perturbed[:, :, i], B_perturbed[i] = proj_tau_B(T, num_edges, tau_perturbed[:, :, i], B_perturbed[i], \
                                                        od_to_edges_list_full, \
                                                        tau_max = tau_upper_limit, B_max = B_upper_limit)
    
    print("tau[:, :, i], after projection:\n", tau[:, :, i])
    print("B[i], after projection:\n", B[i])
    print("tau_perturbed[:, :, i], after projection:\n", tau_perturbed[:, :, i])
    print("B_perturbed[i], after projection:\n", B_perturbed[i])

    # TODO: Remove "network":

    y_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                 tau[:, :, i], B[i], od_to_edges_array, \
                                 demand_array_temp, VoT_array, num_el, coeff_input)

    y_perturbed_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                           tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
                                           demand_array_temp, VoT_array, num_el, coeff_input)
    
#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input):

    welfare, obj_E, obj_R, obj_I = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                                               tau[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                                               y = y_values, coeff_input = coeff_input)
    
    welfare_perturbed, obj_E_perturbed, obj_R_perturbed, obj_I_perturbed \
        = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                      tau_perturbed[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                      y = y_perturbed_values, coeff_input = coeff_input)
    
    print("welfare:", welfare)
    print("obj_E:", obj_E)
    print("obj_R:", obj_R)
    print("obj_I:", obj_I)
    print()
    print("welfare_perturbed:", welfare_perturbed)
    print("obj_E_perturbed:", obj_E_perturbed)
    print("obj_R_perturbed:", obj_R_perturbed)
    print("obj_I_perturbed:", obj_I_perturbed)
    print()
    
    welfare_list.append(welfare)
    
    tau[:, :, i+1] = tau[:, :, i] - eta[i] * (d/delta[i]) * w_i[:-1].reshape((num_edges, T)) \
                        * (welfare_perturbed - welfare)
    
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare)
    
    print("tau[:, :, i+1], before projection:\n", tau[:, :, i+1])
    print("B[i+1], before projection:\n", B[i+1])
    
    tau[:, :, i+1], B[i+1] = proj_tau_B(T, num_edges, tau[:, :, i+1], B[i+1], od_to_edges_list_full, \
                                        tau_max = tau_upper_limit, B_max = B_upper_limit)
    
    print("tau[:, :, i+1], after projection:\n", tau[:, :, i+1])
    print("B[i+1], after projection:\n", B[i+1])
    
    if i >= diffs_num_cols + 2:
        tau_diffs = np.max(np.absolute(tau[:, :, i-diffs_num_cols : i-1] - tau[:, :, i-diffs_num_cols+1 : i]), axis = 2)
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[:, :, 0:10]:", tau[:, :, 0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:\n", tau_diffs)
        print("B_diffs:\n", B_diffs)
        print()
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            print("Within error bound.")
            break

time_2 = time.time()

min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[:, :, argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print()
print("Time:", time_2 - time_1)




Iter: 0
i:
 0
tau[:, :, i], before projection:
 [[1.43005214 0.08807116 1.46437401 1.44885104 1.713734  ]
 [1.50130385 1.32967085 0.46962283 1.27673534 1.38234246]
 [1.3020731  4.79582954 5.74207014 2.61783072 2.23783536]
 [2.093914   1.47765498 4.38447225 2.30415345 6.45119086]
 [5.73796645 4.79949634 7.77507744 8.99153941 6.49169219]
 [0.38194635 0.62067379 0.42047625 0.75315872 0.49085684]
 [0.30617847 0.65877564 0.73466998 0.70760944 0.73109563]]
B[i], before projection:
 72.30540636194843
tau_perturbed[:, :, i], before projection:
 [[1.43051962 0.08840429 1.4648115  1.44860733 1.71456224]
 [1.50087327 1.33060086 0.46904573 1.27570559 1.38237327]
 [1.30247229 4.7966159  5.74245174 2.61826825 2.2370151 ]
 [2.09389459 1.47801226 4.38400896 2.30484876 6.4508752 ]
 [5.73843105 4.79989872 7.77500238 8.9925412  6.49211915]
 [0.38160825 0.62076483 0.42145959 0.75317419 0.49100763]
 [0.30708884 0.65865351 0.73416458 0.70808089 0.73171788]]
B_perturbed[i], before projection:
 72.3059508154

KeyboardInterrupt: 

In [ ]:
# dict_latency_params["Belmont"]

In [ ]:
# B
print("argmin_B:", argmin_B)
print("B[argmin_welfare_list]:", B[argmin_welfare_list])

In [ ]:
index_next_init = argmin_welfare_list
B_next_init = B[index_next_init]
tau_next_init = tau[:, :, index_next_init]

print("index_next_init:", index_next_init)
print("B_next_init:", B_next_init)
print("tau_next_init:", tau_next_init)


In [ ]:
# lambda combinations to try to simulate:
# (1, 1, 1)
# (5, 5, 1)
# (10, 10, 1)
# (5, 10, 1)
# (5, 15, 1)
# (10, 15, 1)
# (10, 20, 1)
# (20, 20, 1)
# (20, 30, 1)
# (20, 40, 1)

In [27]:


# argmin_tau = np.array([[0.7091, 1.8252, 0.3238, 0.1215, 0.1079],
#                        [1.8739, 0.7429, 0.0, 0.9112, 1.8739],
#                        [0.0, 3.0448, 0.0, 0.0, 3.9494],
#                        [2.9504, 2.377, 3.4944, 0.0, 0.0],
#                        [7.2392, 8.4126, 0.0, 6.819, 0.6135],
#                        [0.2836, 0.8496, 0.963, 0.9527, 0.5689],
#                        [0.1346, 0.0, 0.0, 0.0712, 0.5024]])

# argmin_B = 0.1664

In [28]:
argmin_y = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                    argmin_tau, argmin_B, od_to_edges_array, \
                                    demand_array_temp, VoT_array, num_el, coeff_input)

print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)
print("argmin_y:\n")

argmin_y



prob.status: optimal
argmin_tau: [[0.7091 1.8252 0.3238 0.1215 0.1079]
 [1.8739 0.7429 0.     0.9112 1.8739]
 [0.     3.0448 0.     0.     3.9494]
 [2.9504 2.377  3.4944 0.     0.    ]
 [7.2392 8.4126 0.     6.819  0.6135]
 [0.2836 0.8496 0.963  0.9527 0.5689]
 [0.1346 0.     0.     0.0712 0.5024]]
argmin_B: 0.1664
argmin_y:



{(0, 0, 0, 0, 0): 2.5152464150374994,
 (0, 0, 0, 1, 0): 0.001770068932960608,
 (0, 0, 0, 2, 0): 45.26106795901079,
 (0, 1, 0, 0, 0): 1.4403257808567052,
 (0, 1, 0, 1, 0): 0.005349602758903668,
 (0, 1, 0, 2, 0): 26.1186040989791,
 (0, 2, 0, 0, 0): -0.0,
 (0, 2, 0, 1, 0): 115.15744798402402,
 (0, 3, 0, 0, 0): -0.0,
 (0, 3, 0, 1, 0): 134.75871572598555,
 (0, 4, 0, 0, 0): 23.906663396901763,
 (0, 4, 0, 1, 0): 263.37441694622197,
 (0, 0, 0, 0, 1): 1.0291622784122167,
 (0, 0, 0, 1, 1): 0.000721571467413784,
 (0, 0, 0, 2, 1): 46.74820059354508,
 (0, 1, 0, 0, 1): 0.5903313597275824,
 (0, 1, 0, 1, 1): 0.0021984645222479465,
 (0, 1, 0, 2, 1): 26.971749658787374,
 (0, 2, 0, 0, 1): -0.0,
 (0, 2, 0, 1, 1): 115.15744798402402,
 (0, 3, 0, 0, 1): -0.0,
 (0, 3, 0, 1, 1): 134.75871572598555,
 (0, 4, 0, 0, 1): -0.0,
 (0, 4, 0, 1, 1): 287.28108034312373,
 (0, 0, 0, 0, 2): 5.091591534520162,
 (0, 0, 0, 1, 2): 0.003726955318609427,
 (0, 0, 0, 2, 2): 42.68276595329322,
 (0, 1, 0, 0, 2): 2.907989186799585,
 (

In [29]:
# demand_array_temp

# tau_upper_limit

B_upper_limit

# y_perturbed_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
#                                            tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
#                                            demand_array_temp, VoT_array, num_el, coeff_input)

# T
# num_edges
# num_gp_lanes
# B_perturbed[i]
# coeff_input
# od_to_edges_array
# demand_array_temp
# VoT_array
# num_el
# coeff_input

# np.minimum(np.array([1, 2]), 0.5)

# tau_upper_limit
# argmin_welfare_list

157.64553451203628

# Compute Cost Metrics

In [30]:
num_groups = demand_array.shape[1]

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

edge_to_od_dict = {}
for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

argmin_x = {}
for e in range(num_edges):
    for t in range(T):
        argmin_x[(e, 0, t)] = 0
        argmin_x[(e, 0, t)] += sum(argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices)
        argmin_x[(e, 0, t)] += sum(argmin_y[(od, g, e, 0, t)] \
                                   for od in edge_to_od_dict[e] for g in in_indices)
        
        argmin_x[(e, 1, t)] = 0
        argmin_x[(e, 1, t)] += sum(argmin_y[(od, g, e, 2, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices)
        argmin_x[(e, 1, t)] += sum(argmin_y[(od, g, e, 1, t)] \
                                   for od in edge_to_od_dict[e] for g in in_indices)

argmin_y_in_el_total = {}      
for e in range(num_edges):
    for t in range(T):
        argmin_y_in_el_total[(e, 0, t, 'el')] = sum(argmin_y[(od, g, e, 0, t)] + argmin_y[(od, g, e, 1, t)] \
                                                    for od in edge_to_od_dict[e] for g in el_indices)
        
        argmin_y_in_el_total[(e, 1, t, 'el')] = sum(argmin_y[(od, g, e, 2, t)] \
                                                    for od in edge_to_od_dict[e] for g in el_indices)
        
        argmin_y_in_el_total[(e, 0, t, 'in')] = sum(argmin_y[(od, g, e, 0, t)] \
                                                    for od in edge_to_od_dict[e] for g in in_indices)
        
        argmin_y_in_el_total[(e, 1, t, 'in')] = sum(argmin_y[(od, g, e, 1, t)] \
                                                    for od in edge_to_od_dict[e] for g in in_indices)


# y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] for g in el_indice

# argmin_x

argmin_y_in_el_total

{(0, 0, 0, 'el'): 10.826490938576722,
 (0, 1, 0, 'el'): 585.4901849771044,
 (0, 0, 0, 'in'): 837.4206779484864,
 (0, 1, 0, 'in'): 3414.366207746645,
 (0, 0, 1, 'el'): 4.3846190786588215,
 (0, 1, 1, 'el'): 591.9320568432798,
 (0, 0, 1, 'in'): 269.0556725988657,
 (0, 1, 1, 'in'): 3982.7312130962646,
 (0, 0, 2, 'el'): 22.675328908059697,
 (0, 1, 2, 'el'): 573.6413470095841,
 (0, 0, 2, 'in'): 1023.9596259590351,
 (0, 1, 2, 'in'): 3227.8272597360965,
 (0, 0, 3, 'el'): 53.7146281613918,
 (0, 1, 3, 'el'): 542.6020477568866,
 (0, 0, 3, 'in'): 1097.05666755627,
 (0, 1, 3, 'in'): 3154.7302181388613,
 (0, 0, 4, 'el'): 59.12180549109018,
 (0, 1, 4, 'el'): 537.1948704272447,
 (0, 0, 4, 'in'): 1098.6495203201557,
 (0, 1, 4, 'in'): 3153.137365374976,
 (1, 0, 0, 'el'): 3.650451399067628,
 (1, 1, 0, 'el'): 655.3078685124077,
 (1, 0, 0, 'in'): 5.413772890031697,
 (1, 1, 0, 'in'): 4509.858676162272,
 (1, 0, 1, 'el'): 9.256998722827051,
 (1, 1, 1, 'el'): 649.701321032904,
 (1, 0, 1, 'in'): 773.93958796164

In [60]:
coeff_input

array([[8.61988500e+02, 1.00151786e+03, 8.81184667e+02, 1.27894812e+03,
        1.03409283e+03, 8.45150000e+02, 8.53181818e+02],
       [7.82665930e-04, 5.84484148e-04, 1.56372403e-03, 1.99413836e-03,
        2.14726170e-03, 3.06601388e-04, 3.21856457e-04],
       [1.32644825e+00, 2.21312655e+00, 4.89219237e+00, 1.19991118e+00,
        5.54100628e+00, 1.50311135e+00, 2.38432845e+00]])

In [68]:
travel_times = {}

## coeff_input: const, slope, x-coordinate of transition point
# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))

for e in range(num_edges):
    for k in range(2):
        for t in range(T):
            travel_times[(e, k, t)] = coeff_input[2, e] + coeff_input[1, e] * max(argmin_x[(e, k, t)] - coeff_input[0, e], 0.0)

edge_demand = {}
avg_travel_time = {}
percent_on_express = {}
obj_E = np.zeros(num_edges)
obj_I = np.zeros(num_edges)
obj_R = np.zeros(num_edges)
obj = np.zeros(num_edges)

for e in range(num_edges):
    avg_travel_time[e, 'el'] = 0.0
    avg_travel_time[e, 'in'] = 0.0
    avg_travel_time[e, 'ex'] = 0.0
    avg_travel_time[e, 'gp'] = 0.0
    
    percent_on_express[e, 'el'] = 0.0
    percent_on_express[e, 'in'] = 0.0
    percent_on_express[e, 'all'] = 0.0
    
    obj_E[e] = 0.0
    obj_I[e] = 0.0
    obj_R[e] = 0.0
    obj[e] = 0.0
    
    for t in range(T):
        edge_demand[e, t, 'el'] = sum(argmin_y_in_el_total[(e, k, t, 'el')] for k in range(2))
        edge_demand[e, t, 'in'] = sum(argmin_y_in_el_total[(e, k, t, 'in')] for k in range(2))
        
    percent_on_express[e, 'el'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] for t in range(T))
    percent_on_express[e, 'in'] += sum(argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'in'] for t in range(T))
    percent_on_express[e, 'all'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] + edge_demand[e, t, 'in'] for t in range(T))
    
    avg_travel_time[e, 'el'] += sum(argmin_y_in_el_total[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand[e, t, 'el'] for t in range(T))    
    avg_travel_time[e, 'in'] += sum(argmin_y_in_el_total[(e, k, t, 'in')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand[e, t, 'in'] for t in range(T))    
    avg_travel_time[e, 'ex'] += sum( (argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')]) * travel_times[(e, 0, t)] for t in range(T)) \
                                    / sum( argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T))
    avg_travel_time[e, 'gp'] += sum( (argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')]) * travel_times[(e, 0, t)] for t in range(T)) \
                                    / sum( argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')] for t in range(T))
    
    obj_E[e] = sum( argmin_y[(od, g, e, 0, t)] * VoT_array[od, g] * travel_times[e, 0, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g] * travel_times[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g] * travel_times[e, 1, t] \
                          for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 

    obj_I[e] = sum( argmin_y[(od, g, e, 0, t)] * (VoT_array[od, g] * travel_times[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * VoT_array[od, g] * travel_times[e, 1, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

    obj_R[e] = sum( argmin_y[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( argmin_y[(od, g, e, 1, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) )

    obj[e] = lambda_E * obj_E[e] - lambda_R * obj_R[e] + lambda_I * obj_I[e]


# welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, argmin_tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input)

# avg_travel_time
# percent_on_express
# obj_R



In [69]:
# e = 0
# od = 0
# g = 0
# t = 0

# # sum( argmin_y[(od, g, e, 0, t)] * VoT_array[od, g] * travel_times[e, 0, t] \
# #                         for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
# sum( argmin_y[(od, g, e, 1, t)] * (VoT_array[od, g] * travel_times[e, 0, t] + tau[e, t]) \
#     for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
# #                     + sum( argmin_y[(od, g, e, 2, t)] * VoT_array[od, g] * travel_times[e, 1, t] \
# #                           for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) ) 

# # argmin_y[(od, g, e, 1, t)]



# # sum(edge_demand[e, t, 'el'] for t in range(T) )

In [70]:
travel_times

{(0, 0, 0): 1.326448252485291,
 (0, 0, 1): 1.326448252485291,
 (0, 0, 2): 1.4709647418103666,
 (0, 0, 3): 1.5524687078719717,
 (0, 0, 4): 1.5579473929353962,
 (0, 1, 0): 3.782350545051435,
 (0, 1, 1): 4.232232304194485,
 (0, 1, 2): 3.627079184032555,
 (0, 1, 3): 3.5455752179714466,
 (0, 1, 4): 3.5400965329080663,
 (1, 0, 0): 2.21312655334384,
 (1, 0, 1): 2.21312655334384,
 (1, 0, 2): 2.3848560077973997,
 (1, 0, 3): 2.21312655334384,
 (1, 0, 4): 2.21312655334384,
 (1, 1, 0): 4.6467132097616775,
 (1, 1, 1): 4.194245115263921,
 (1, 1, 2): 3.894910384849517,
 (1, 1, 3): 4.262109100652053,
 (1, 1, 4): 4.646713222663328,
 (2, 0, 0): 6.1715734803783056,
 (2, 0, 1): 4.944197455240471,
 (2, 0, 2): 6.1715734803783056,
 (2, 0, 3): 6.1715734803783056,
 (2, 0, 4): 4.892192374894639,
 (2, 1, 0): 11.480451984637075,
 (2, 1, 1): 12.707827853282256,
 (2, 1, 2): 11.480451984637075,
 (2, 1, 3): 11.480451984637075,
 (2, 1, 4): 13.07338313429418,
 (3, 0, 0): 1.1999111785887613,
 (3, 0, 1): 1.19991117858876

In [71]:
opt_data_array = np.zeros((num_edges, 16))

# argmin_tau
opt_data_array[:, 0:5] = argmin_tau

# argmin_tau_avg
opt_data_array[:, 5] = np.linalg.norm(argmin_tau, axis=1)

# B (as array)
opt_data_array[:, 6] = argmin_B * np.eye(1, num_edges, 0)

# percent_on_express (overall)
# percent_on_express (eligible)
# percent_on_express (ineligible)
opt_data_array[:, 7] = np.array([percent_on_express[e, 'all'] for e in range(num_edges)])
opt_data_array[:, 8] = np.array([percent_on_express[e, 'el'] for e in range(num_edges)])
opt_data_array[:, 9] = np.array([percent_on_express[e, 'in'] for e in range(num_edges)])

# avg_travel_time (express lane)
# avg_travel_time (general purpose lane)
opt_data_array[:, 10] = np.array([avg_travel_time[e, 'ex'] for e in range(num_edges)])
opt_data_array[:, 11] = np.array([avg_travel_time[e, 'gp'] for e in range(num_edges)])

# obj_E = {}
# obj_I = {}
# obj_R = {}
# obj
opt_data_array[:, 12] = np.array([obj_E[e] for e in range(num_edges)]) 
opt_data_array[:, 13] = np.array([obj_I[e] for e in range(num_edges)]) 
opt_data_array[:, 14] = np.array([obj_R[e] for e in range(num_edges)]) 
opt_data_array[:, 15] = np.array([obj[e] for e in range(num_edges)]) 


In [72]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["tau (time-averaged)", \
                 "B", \
                 "% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (express lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_opt_save = pd.DataFrame(opt_data_array, index=row_names, columns=column_names)

# df_opt_save



In [73]:
random_string = ""
for idx_rand_str in range(20):
    random_string += str(np.random.randint(1, 9))

directory_to_save = "../data/opt_tolls_subsidies_metrics/"
random_filename = "opt___" + random_string + '.csv'

df_opt_save.to_csv(directory_to_save + random_filename)

# Compute good initializations for alpha:

In [95]:
# # el_indices
# # in_indices

# e = 0
# # VoT_array[edge_to_od_dict[e], in_indices]

# VoT_array_trunc_in_temp = VoT_array[edge_to_od_dict[e], :]
# VoT_array_trunc_in = VoT_array_trunc_in_temp[:, in_indices]

# # print("VoT_array[:, in_indices]:\n", VoT_array[:, in_indices])
# # print()
# # print("VoT_array[edge_to_od_dict[e], :]:\n", VoT_array[edge_to_od_dict[e], :])

# # VoT_array_trunc_in

array([[0.27544071, 0.58092949, 1.85964494],
       [0.27544071, 0.58092949, 1.85964494],
       [0.27544071, 0.58092949, 1.85964494],
       [0.27544071, 0.58092949, 1.85964494],
       [0.27544071, 0.58092949, 1.85964494],
       [0.27544071, 0.58092949, 1.85964494],
       [0.27544071, 0.58092949, 1.85964494]])

In [97]:
# demand_array.shape

(24, 5)

In [107]:
arr_temp = np.stack([np.array([1, 2, 3]), np.array([3, 1, 4])]).T
arr_temp[0:, :]

array([[1, 3],
       [2, 1],
       [3, 4]])

In [168]:
alpha_init = np.zeros((num_edges, T))

for e in range(num_edges):
#     print("edge_to_od_dict[e]:", edge_to_od_dict[e])

    ## Ineligible users:
    
    VoT_array_trunc_in_temp = VoT_array[edge_to_od_dict[e], :]
    VoT_array_trunc_in = VoT_array_trunc_in_temp[:, in_indices]
    demand_array_trunc_in_temp = demand_array[edge_to_od_dict[e], :]
    demand_array_trunc_in = demand_array_trunc_in_temp[:, in_indices]
    
    VoT_array_trunc_in_flattened = VoT_array_trunc_in.flatten()
    demand_array_trunc_in_flattened = demand_array_trunc_in.flatten()
    VoT_demand_in = np.stack([VoT_array_trunc_in_flattened, demand_array_trunc_in_flattened]).T    
    VoT_demand_in_sorted = VoT_demand_in[np.argsort(VoT_demand_in[:, 0])]
    VoT_demand_in_cumul = np.zeros(VoT_demand_in_sorted.shape)
    VoT_demand_in_cumul[:, 0] = VoT_demand_in_sorted[:, 0]
    for row_index in range(VoT_demand_in_cumul.shape[0]):
        VoT_demand_in_cumul[row_index, 1] = np.sum(VoT_demand_in_sorted[row_index:, 1])

    ## Eligible users:
    
    # Flatten VoT_array_trunc_in and demand_array_trunc_in, and concatenate.
    # Sort by VoT
    # Create cumulative demand array
    
    VoT_array_trunc_el_temp = VoT_array[edge_to_od_dict[e], :]
    VoT_array_trunc_el = VoT_array_trunc_el_temp[:, el_indices]
    demand_array_trunc_el_temp = demand_array[edge_to_od_dict[e], :]
    demand_array_trunc_el = demand_array_trunc_el_temp[:, el_indices]
    
    VoT_array_trunc_el_flattened = VoT_array_trunc_el.flatten()
    demand_array_trunc_el_flattened = demand_array_trunc_el.flatten()
    VoT_demand_el = np.stack([VoT_array_trunc_el_flattened, demand_array_trunc_el_flattened]).T    
    VoT_demand_el_sorted = VoT_demand_el[np.argsort(VoT_demand_el[:, 0])]
    VoT_demand_el_cumul = np.zeros(VoT_demand_el_sorted.shape)
    VoT_demand_el_cumul[:, 0] = VoT_demand_el_sorted[:, 0]
    for row_index in range(VoT_demand_el_cumul.shape[0]):
        VoT_demand_el_cumul[row_index, 1] = np.sum(VoT_demand_el_sorted[row_index:, 1])
    
#     VoT_array[od, g]

VoT_in_boundary = np.zeros((num_edges, T))
VoT_el_boundary = np.zeros((num_edges, T))

for e in range(num_edges):
    for t in range(T):
        
# for e in [0]:
#     for t in [0]:
        
        print()
        print("e:", e)
        print("t:", t)
        
        ## Ineligible users:
        
        # Find minimum entry in cumulative demand array such that demand >= argmin_y_in_el_total[(e, 0, t, 'in')]
        # Find corresponding VoT index -> Value
        
        VoT_in_boundary_index = max([row_index for row_index in range(VoT_demand_in_cumul.shape[0]) \
                                     if VoT_demand_in_cumul[row_index, 1] >= argmin_y_in_el_total[(e, 0, t, 'in')]])
        VoT_in_boundary[e, t] = VoT_demand_in_cumul[VoT_in_boundary_index, 0]
        
        ## Eligible users:
        
        # Find minimum entry in cumulative demand array such that demand >= argmin_y_in_el_total[(e, 0, t, 'in')]
        # Find corresponding VoT index -> Value
        
        VoT_el_boundary_index = max([row_index for row_index in range(VoT_demand_el_cumul.shape[0]) \
                                     if VoT_demand_el_cumul[row_index, 1] >= argmin_y_in_el_total[(e, 0, t, 'el')]])
        VoT_el_boundary[e, t] = VoT_demand_el_cumul[VoT_el_boundary_index, 0]
        
        alpha_init[e, t] = 1.0 - VoT_el_boundary[e, t]/VoT_in_boundary[e, t]
        
        print("VoT_in_boundary_index:", VoT_in_boundary_index)
        print("VoT_el_boundary_index:", VoT_el_boundary_index)
        print("VoT_in_boundary[e, t]:", VoT_in_boundary[e, t])
        print("VoT_el_boundary[e, t]:", VoT_el_boundary[e, t])
        print("alpha_init[e, t]:", alpha_init[e, t])
        
        assert 0.0 <= alpha_init[e, t] <= 1.0



e: 0
t: 0
VoT_in_boundary_index: 19
VoT_el_boundary_index: 13
VoT_in_boundary[e, t]: 1.7585202338411385
VoT_el_boundary[e, t]: 0.1081094831094831
alpha_init[e, t]: 0.9385224684771813

e: 0
t: 1
VoT_in_boundary_index: 20
VoT_el_boundary_index: 13
VoT_in_boundary[e, t]: 1.8596449415012848
VoT_el_boundary[e, t]: 0.1081094831094831
alpha_init[e, t]: 0.9418655245972887

e: 0
t: 2
VoT_in_boundary_index: 19
VoT_el_boundary_index: 13
VoT_in_boundary[e, t]: 1.7585202338411385
VoT_el_boundary[e, t]: 0.1081094831094831
alpha_init[e, t]: 0.9385224684771813

e: 0
t: 3
VoT_in_boundary_index: 19
VoT_el_boundary_index: 13
VoT_in_boundary[e, t]: 1.7585202338411385
VoT_el_boundary[e, t]: 0.1081094831094831
alpha_init[e, t]: 0.9385224684771813

e: 0
t: 4
VoT_in_boundary_index: 19
VoT_el_boundary_index: 12
VoT_in_boundary[e, t]: 1.7585202338411385
VoT_el_boundary[e, t]: 0.10471299533799534
alpha_init[e, t]: 0.9404539149889276

e: 1
t: 0
VoT_in_boundary_index: 20
VoT_el_boundary_index: 13
VoT_in_boundary[

In [169]:
# VoT_array_trunc_in_flattened
# demand_in_trunc_in_flattened

print("argmin_y_in_el_total[(e, 0, t, el)]:\n", argmin_y_in_el_total[(e, 0, t, 'el')])
print()
print("argmin_y_in_el_total[(e, 0, t, in)]:\n", argmin_y_in_el_total[(e, 0, t, 'in')])
print()
print("VoT_demand_el_cumul:\n", VoT_demand_el_cumul)
print()
print("VoT_demand_in_cumul:\n", VoT_demand_in_cumul)



argmin_y_in_el_total[(e, 0, t, el)]:
 30.01047036405467

argmin_y_in_el_total[(e, 0, t, in)]:
 1086.6913551772764

VoT_demand_el_cumul:
 [[2.90464744e-02 8.15841988e+02]
 [2.92776134e-02 7.62446356e+02]
 [3.14503205e-02 7.16857885e+02]
 [3.17457084e-02 6.32110480e+02]
 [3.19742357e-02 6.11616911e+02]
 [3.21791462e-02 4.92193733e+02]
 [3.50560897e-02 4.75125463e+02]
 [9.32921245e-02 4.18445197e+02]
 [9.38644689e-02 3.93024370e+02]
 [9.82216708e-02 3.06771791e+02]
 [1.02386040e-01 2.01685010e+02]
 [1.02436626e-01 1.32787023e+02]
 [1.04712995e-01 1.02220342e+02]
 [1.08109483e-01 5.52321865e+01]]

VoT_demand_in_cumul:
 [[2.57822154e-01 6.29756911e+03]
 [2.60889121e-01 6.18224936e+03]
 [2.63667482e-01 5.99638867e+03]
 [2.64868234e-01 5.60994051e+03]
 [2.65190660e-01 5.55764623e+03]
 [2.71164457e-01 5.29395978e+03]
 [2.75440705e-01 5.06008691e+03]
 [5.67805040e-01 4.77224643e+03]
 [5.69931498e-01 4.67151533e+03]
 [5.80929487e-01 4.44795648e+03]
 [5.89432300e-01 4.11112187e+03]
 [5.91529407e-

In [170]:
# argmin_tau
# alpha_init

init_tau_alpha_array = np.block([argmin_tau, alpha_init])
init_tau_alpha_array

array([[0.7091    , 1.8252    , 0.3238    , 0.1215    , 0.1079    ,
        0.93852247, 0.94186552, 0.93852247, 0.93852247, 0.94045391],
       [1.8739    , 0.7429    , 0.        , 0.9112    , 1.8739    ,
        0.94186552, 0.93852247, 0.93614605, 0.94186552, 0.94186552],
       [0.        , 3.0448    , 0.        , 0.        , 3.9494    ,
        0.93614605, 0.93852247, 0.93614605, 0.93614605, 0.94186552],
       [2.9504    , 2.377     , 3.4944    , 0.        , 0.        ,
        0.93852247, 0.93852247, 0.93852247, 0.93614605, 0.93614605],
       [7.2392    , 8.4126    , 0.        , 6.819     , 0.6135    ,
        0.94186552, 0.94186552, 0.93546203, 0.94186552, 0.93546203],
       [0.2836    , 0.8496    , 0.963     , 0.9527    , 0.5689    ,
        0.93614605, 0.94186552, 0.94186552, 0.94186552, 0.93852247],
       [0.1346    , 0.        , 0.        , 0.0712    , 0.5024    ,
        0.93952656, 0.93614605, 0.93614605, 0.94198621, 0.93852247]])

In [171]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["alpha (t=" + str(t+1) + ")" for t in range(T) ]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_inits_save = pd.DataFrame(init_tau_alpha_array, index=row_names, columns=column_names)

# df_inits_save

random_string = ""
for idx_rand_str in range(10):
    random_string += str(np.random.randint(1, 9))

directory_to_save = "../data/opt_tolls_subsidies_metrics/"
random_filename = "inits___" + random_string + '.csv'

df_inits_save.to_csv(directory_to_save + random_filename)

In [ ]:
# argmin_y_in_el_total[(e, 0, t, 'in')]

In [130]:
# demand_array_trunc_in_flattened

# Testing solve CBCP direct with a small example:

In [ ]:
# od_to_edges_array = np.array([[0, 0], [0, 1], [1, 1], [1, 2]])
# od_to_edges_list_full = [[0], [0, 1], [1], [1, 2]]
# edge_to_ods = [[0, 1], [1, 2, 3], [3]]

# num_od = 3
# num_edges = 3
# num_gp_lanes = 3
# num_groups = 5
# T = 5
# el_indices = [0, 1]
# in_indices = [2, 3, 4]
# B = 3
# tau = np.zeros((num_edges, T))

# for e in range(num_edges):
#     for t in range(T):
#         tau[e, t] = 1.5 + 0.5 * e + 0.2 * t

# demand_array = np.zeros((num_od, num_groups))
# VoT_array = np.zeros((num_od, num_groups))

# for od in range(num_od):
#     for g in range(num_groups):
#         demand_array[od, g] = 1.0 + od + 0.1 * g
#         VoT_array[od, g] = 2.0 + od + 0.1 * g
        
# coeff_input = np.array([1, 1, 0, 0, 0])
# latency_params_length = coeff_input.shape[0]

# ex_to_gp_multiplier = np.array([1/num_gp_lanes**p for p in range(latency_params_length)]).reshape((latency_params_length, 1)) \
#                         @ np.ones((1, num_edges))
# a = np.zeros((latency_params_length, num_edges, 2))
# coeff[:, :, 0] = coeff_input.reshape((latency_params_length, 1)) @ np.ones((1, num_edges))
# coeff[:, :, 1] = coeff[:, :, 0] * ex_to_gp_multiplier


# y_vals = solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                            demand_array, VoT_array, num_el, coeff_input)


# # y = {}
# # for od in range(num_od):
# #     for e in od_to_edges_list_full[od]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 for k in [0, 1, 2]:
# #                     y[(od, g, e, k, t)] = cp.Variable(1)
# #             for g in in_indices:
# #                 for k in [0, 1]:
# #                     y[(od, g, e, k, t)] = cp.Variable(1)

# # x = {}
# # for e in range(num_edges):
# #     for k in [0, 1]:
# #         for t in range(T):
# #             x[(e, k, t)] = cp.Variable(1)

# # # Objective:
# # func = 0.0
# # for e in range(num_edges):

# # for od in range(num_od):
# #     for e in od_to_edges_list_full[od]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g]
# #             for g in in_indices:
# #                 func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g]

# # objective = cp.Minimize(func)

# # # Constraints:
# # constraints = []

# # constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
# #                 for e in od_to_edges_list_full[od] for g in el_indices  \
# #                 for k in [0, 1, 2] for t in range(T)]
# # constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
# #                 for e in od_to_edges_list_full[od] for g in in_indices  \
# #                 for k in [0, 1] for t in range(T)]


# # for e in range(num_edges):
# #     for t in range(T):
# # #         print("e:", e)
# # #         print("edge_to_ods[e]:", edge_to_ods[e])

# #         ## Edge contraints:
# #         constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
# #                             + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
# #                             == x[(e, 0, t)] ]
# #         constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
# #                             + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
# #                             == x[(e, 1, t)] ]

# #         ## Group flow constraints:
# #         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
# #                         for od in edge_to_ods[e] for g in el_indices]
# #         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
# #                         for od in edge_to_ods[e] for g in in_indices]

# # constraints += [sum(y[(od, g, e, 0, t)] * tau[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
# #                 <= B * demand_array[od, g] for od in range(num_od) for g in el_indices]

# # # Problem:
# # prob = cp.Problem(objective, constraints)

# # # Solve:
# # result = prob.solve()


# # assert prob.status != "infeasible", "problem.status should not be infeasible."
# # assert prob.status != "unbounded", "problem.status should not be unbounded."
# # print()
# # print("prob.status:", prob.status)

# # # Extract Values:
# # y_values = {}
# # for e in range(num_edges):
# #     for od in edge_to_ods[e]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 for k in [0, 1, 2]:
# #                     print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
# #                     y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
# #             for g in in_indices:
# #                 for k in [0, 1]:
# #                     print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
# #                     y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)




In [ ]:
# demand_edges_array

In [ ]:
# np.sum(demand_array[3:16, :])/5

In [ ]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

## Test:

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:
func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)


## Linear Approximation for Latency Function:

In [ ]:
# Variables:
v = cp.Variable(1)
            
# Objective:
func = v - 1 + cp.square(cp.maximum(v-1, 0))
objective = cp.Minimize(func)

# Constraints:
constraints = [-3.0 <= v, v <= 3.0]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("v.value:", v.value)
